In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("salary-train.csv")

In [3]:
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [7]:
data['FullDescription'] = data['FullDescription'].apply(lambda s : s.lower())
data['LocationNormalized'] = data['LocationNormalized'].apply(lambda s : s.lower())

In [8]:
data['FullDescription'] = data['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
data['LocationNormalized'] = data['LocationNormalized'].replace('[^a-zA-Z0-9]',' ', regex = True)

In [12]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
enc = TfidfVectorizer(min_df = 5)
data_categ = enc.fit_transform(data['FullDescription'])

In [17]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)

In [19]:
dict_vect = DictVectorizer()
data_categ2 = dict_vect.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [21]:
import scipy.sparse

full_data = scipy.sparse.hstack([data_categ, data_categ2])

In [22]:
from sklearn.linear_model import Ridge

In [23]:
model = Ridge(alpha = 1)
model.fit(full_data, data['SalaryNormalized'])

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, solver='auto', tol=0.001)

In [24]:
test = pd.read_csv("salary-test-mini.csv")

In [25]:
test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [29]:
test['FullDescription'] = test['FullDescription'].apply(lambda s : s.lower())
test['LocationNormalized '] = test['LocationNormalized'].apply(lambda s : s.lower())

test_data1 = enc.transform(test['FullDescription'])
test_data2 = dict_vect.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [32]:
preds = model.predict(scipy.sparse.hstack([test_data1, test_data2]))

In [35]:
"%.2f" % preds[0]

'55880.55'

In [36]:
"%.2f" % preds[1]

'37180.35'

In [43]:
data = pd.read_csv("close_prices.csv")
data.drop('date', inplace = True, axis = 1)
data.head()

,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,75.910004,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,76.040001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,75.519997,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,76.070000,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,75.959999,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002


In [44]:
import sklearn.decomposition

In [45]:
model = sklearn.decomposition.PCA(n_components=10)
model.fit(data)

PCA(copy=True, n_components=10, whiten=False)

In [48]:
np.cumsum(model.explained_variance_ratio_) # 4

array([ 0.73897118,  0.84904287,  0.89899376,  0.92774295,  0.94989743,
        0.9692132 ,  0.97596173,  0.98210264,  0.98530858,  0.98836469])

In [52]:
ff = pd.DataFrame(model.transform(data))

In [53]:
ff[0:10]

,0,1,2,3,4,5,6,7,8,9
0,-50.902404,-17.631677,-7.736021,2.596010,2.377569,4.244055,-12.883088,3.556570,-5.821974,-1.726040
1,-52.846909,-19.146907,-7.272546,4.618575,0.102182,3.935519,-11.062930,3.432599,-5.633181,-2.012232
2,-54.614439,-19.927358,-8.109958,4.920347,-0.421454,4.555335,-10.119224,4.175203,-5.388287,-2.752699
3,-52.600566,-19.941571,-6.948113,4.806243,0.319779,5.065505,-9.849605,3.266579,-5.974574,-1.922746
4,-52.370123,-22.289064,-8.410868,3.833372,-2.464882,4.359897,-7.988691,3.452709,-5.745204,-1.875131
5,-54.653412,-24.190939,-10.139937,3.492786,-4.591832,4.886501,-6.940188,4.586919,-4.882303,-1.785527
6,-52.812575,-23.005650,-9.614647,2.953916,-3.605376,6.053104,-7.250570,4.231228,-4.860103,-1.150869
7,-53.651146,-23.884581,-11.453830,3.563354,-4.270606,5.896830,-6.119343,5.237782,-3.870597,-0.171450
8,-56.692727,-25.622408,-14.305737,3.039053,-7.108592,6.942510,-5.332325,4.918475,-3.151208,-0.327393
9,-54.402655,-24.754977,-12.272721,3.548340,-6.778208,6.684524,-4.653229,4.672059,-3.051907,0.247985


In [54]:
first_col = ff[ff.columns[0]]

In [58]:
dj = pd.read_csv("djia_index.csv")
dj.head()

,date,^DJI
0,2013-09-23,15401.379883
1,2013-09-24,15334.589844
2,2013-09-25,15273.259766
3,2013-09-26,15328.299805
4,2013-09-27,15258.240234


In [62]:
'%.2f' % np.corrcoef(first_col, dj['^DJI'])[0][1]

'0.91'

In [68]:
np.argmax(model.components_[0])

26

In [69]:
data.columns[26]

'V'